In [78]:
import os
from pyspark.sql import SparkSession

In [2]:
# 创建sparkSession对象
spark = SparkSession \
    .builder \
    .appName("myApp") \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/bigdata.raw") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/bigdata.raw") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.3.1')\
    .getOrCreate()

In [3]:
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()
df.show()

+--------------------+--------------------+--------------------+
|                 _id|               emoji|            sentence|
+--------------------+--------------------+--------------------+
|[5ea4d816a1efd41d...|      :red_heart:,18|No object is so b...|
|[5ea4d816a1efd41d...|:person_shrugging...|Cant expect diffe...|
|[5ea4d816a1efd41d...|:face_with_tears_...|“ Lets go Marcus ...|
|[5ea4d816a1efd41d...|:face_with_tears_...|Asahd really is a...|
|[5ea4d816a1efd41d...|:face_with_tears_...|Yoongi Tweet Hell...|
|[5ea4d816a1efd41d...|:backhand_index_p...|we cannot afford ...|
|[5ea4d816a1efd41d...|:party_popper:,8 ...|ranks 6th in Janu...|
|[5ea4d816a1efd41d...|:person_facepalmi...|Ok people are rea...|
|[5ea4d816a1efd41d...|:smiling_face_wit...|Cant wait to meet...|
|[5ea4d816a1efd41d...| :clapping_hands:,11|Congratulations M...|
|[5ea4d816a1efd41d...|:face_with_tears_...|Met orlando brown...|
|[5ea4d816a1efd41d...|      :weary_face:,4|Im goin to bed :w...|
|[5ea4d816a1efd41d...|  :

## Find the appearance frequency of every emoji.

In [39]:
# emoji = ":red_heart:"
emojis = df.select('emoji')
# emojis.show()
def split_str(line):
    res = []
    
    words = line.emoji.split(" ")
    for word in words:
        tmp = word.split(',')[0]
        res.append(tmp)
    return " ".join(res)

emojis = emojis.rdd.map(split_str)
emojis.take(3)

[':red_heart:', ':person_shrugging: :female_sign:', ':face_with_tears_of_joy:']

In [42]:
from operator import add

result = emojis.flatMap(lambda x: x.split(" ")) \
        .map(lambda x: (x, 1)) \
        .reduceByKey(add).take(10)
#         sortBy(lambda x: x[1], False)
# result.take(3)
for v, k in result:
    print("{} {}".format(v, k))

:face_with_tears_of_joy: 306649
:backhand_index_pointing_down: 19612
:loudly_crying_face: 122923
 57819
:smiling_face: 17496
:heart_suit: 25808
:face_with_rolling_eyes: 25730
:eyes: 27050
:right_arrow: 17540
:OK_hand: 19396


## For every emoji, ﬁnd the 3 other emojis that are used most frequently with it.

In [14]:
from itertools import combinations

emojis = df.select('emoji')

def split_arr(line):
    res = []
    
    words = line.emoji.split(" ")
    for word in words:
        tmp = word.split(',')[0]
        res.append(tmp)
    return res

def combination(line):
    combs = list(combinations(line, 2))
    res = []
    for comb in combs:
        res.append(str(comb))
    return 
result = emojis.rdd.map(split_arr) \
        .filter(lambda x: len(x) > 1) \
        .map(combination) \
        .map(lambda x: " ".join(x)) \
#         .flatMap(lambda x: x.split(" ")) \
#         .map(lambda x: (x, 1)). \
#         reduceByKey(add).take(10)
result.take(3)
# for v, k in result:
#     print("{} {}".format(v, k))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 14.0 failed 1 times, most recent failure: Lost task 0.0 in stage 14.0 (TID 25, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/opt/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/anaconda3/lib/python3.7/site-packages/pyspark/rdd.py", line 1354, in takeUpToNumLeft
    yield next(iterator)
  File

## For every emoji, determine it is used more with words begin with lower case or word begin with upper case.

In [103]:
def check_case(line):
    res = []
    sentence = line.sentence.split(' ')
    emojis = line.emoji.split(' ')
    
    for emoji in emojis:
        e, place = emoji.split(',')
        word = sentence[int(place)-1]

        if word[0].isupper():
            res.append((e, 1))
        elif word[0].islower():
            res.append((e, -1))
        else:
            res.append((e, 0))
    if len(res) > 0:
        return res
    else:
        return "error"

result = df.rdd.map(check_case) \
#         .flatMap(lambda x: x) \
#         .reduceByKey(add)

result.take(22)

[[(':red_heart:', 1)],
 [(':person_shrugging:', -1), (':female_sign:', 0)],
 [(':face_with_tears_of_joy:', 1)],
 [(':face_with_tears_of_joy:', -1)],
 [(':face_with_tears_of_joy:', -1)],
 [(':backhand_index_pointing_down:', 1)],
 [(':party_popper:', 1), (':backhand_index_pointing_down:', -1)],
 [(':person_facepalming:', -1), (':female_sign:', 0)],
 [(':smiling_face_with_heart-eyes:', -1)],
 [(':clapping_hands:', 1)],
 [(':face_with_tears_of_joy:', -1)],
 [(':weary_face:', -1)],
 [(':clapping_hands:', -1)],
 [(':person_shrugging:', -1), (':male_sign:', 0), (':loudly_crying_face:', 0)],
 [(':male_sign:', 0)],
 [(':folded_hands:', -1)],
 [(':party_popper:', 1)],
 [(':hundred_points:', 1)],
 [(':loudly_crying_face:', -1)],
 [(':rolling_on_the_floor_laughing:', -1)],
 [(':loudly_crying_face:', -1)],
 [(':flexed_biceps:', 1)]]

In [105]:
def check_case(line):
    return line

result = df.rdd.map(check_case) \
#         .flatMap(lambda x: x) \
#         .reduceByKey(add)

result.take(23)

[Row(_id=Row(oid='5ea4d816a1efd41df223a5e1'), emoji=':red_heart:,18', sentence='No object is so beautiful that under certain conditions it will not look ugly Oscar Wilde ↺ RT :red_heart: …'),
 Row(_id=Row(oid='5ea4d816a1efd41df223a5e2'), emoji=':person_shrugging:,13 :female_sign:,15', sentence='Cant expect different results doing the same thingdoing stuff different from now on :person_shrugging: 🏻 \u200d :female_sign: ️'),
 Row(_id=Row(oid='5ea4d816a1efd41df223a5e3'), emoji=':face_with_tears_of_joy:,14', sentence='“ Lets go Marcus ” “ Shiiit where we goin Home ” Marcus Peters :face_with_tears_of_joy:'),
 Row(_id=Row(oid='5ea4d816a1efd41df223a5e4'), emoji=':face_with_tears_of_joy:,14', sentence='Asahd really is a grown man in the body of a 1 year old :face_with_tears_of_joy:'),
 Row(_id=Row(oid='5ea4d816a1efd41df223a5e5'), emoji=':face_with_tears_of_joy:,10', sentence='Yoongi Tweet Hello Im Min fell on Butt What the :face_with_tears_of_joy: Min'),
 Row(_id=Row(oid='5ea4d816a1efd41df223a